In [2]:
import pandas as pd
import pyxirr
import yfinance as yf
import numpy as np
from enum import Enum

In [3]:
INVEST_FILE = 'G:\My Drive\invest\investment_returns.xlsx'
STOCK_FILE = 'G:\My Drive\invest\stock_purchase.xlsx'
SHEETS_TO_SOURCE_MAP = {"Saxo - SGD": "saxo", 
                        "IBKR - SGD(U7470748)": "ibkr",
                         "IBKR endowment plan": "ibkr_endowment", 
                         "Tiger broker": "tiger", 
                         "philips - SGD": "philips"}

In [84]:
def remove_unnamed_columns(dfs: dict):
    for key in dfs.keys():
        dfs[key] = dfs[key].loc[:, ~dfs[key].columns.str.contains('^Unnamed')]

def filter_invalid_timestamps(dfs: dict):
    for key, df in dfs.items():
        df['parse_timestamp'] = pd.to_datetime(df['Date'], errors='coerce')
        invalid_rows = df[df['parse_timestamp'].isna()]
        df.drop(invalid_rows.index, inplace=True)
        df['date'] = df['parse_timestamp'].dt.date
        df.drop(columns=['parse_timestamp', 'Date'], inplace=True)
        dfs[key] = df
        # print(f"dropped invalid rows in {key}")
        # print(invalid_rows)

def rename_columns(dfs: dict):
    for key, df in dfs.items():
        df.columns = [col.lower() for col in df.columns]
        df = df[['date', 'source', 'amount', 'desc']]
        dfs[key] = df

def get_sp500_data(start_date: pd.Timestamp, end_date: pd.Timestamp):
    sp500_ticker = '^GSPC'
    sp500_data = yf.download(sp500_ticker, start=start_date, end=end_date + pd.Timedelta(days=1), progress=False, auto_adjust=True)
    assert sp500_data is not None
    sp500_data.columns = sp500_data.columns.get_level_values(0)
    sp500_data.columns.name = None
    sp500_data.index = pd.to_datetime(sp500_data.index, errors='coerce').map(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
    return sp500_data

def calculate_my_returns() -> pd.DataFrame:
    sheets = list(SHEETS_TO_SOURCE_MAP.keys())
    dfs = {sheet : pd.read_excel(INVEST_FILE, sheet_name=sheet) for sheet in sheets}
    for sheet, source in SHEETS_TO_SOURCE_MAP.items():
        dfs[sheet]["source"] = source
    
    remove_unnamed_columns(dfs)
    filter_invalid_timestamps(dfs)
    rename_columns(dfs)

    df = pd.concat(dfs.values(), axis=0)
    df.date = pd.to_datetime(df.date, errors='coerce')
    df.sort_values(by='date', ascending=True, inplace=True)

    df_curr = df[df['desc'] == 'current value'].copy()
    df_other = df[df['desc'] != 'current value'].copy()

    df_other['cumulative_investment'] = -df_other['amount'].cumsum()
    df_curr['cumulative_investment'] = df_other['cumulative_investment'].iloc[-1]
    
    idx = df_curr['date'].idxmax()
    amount_sum = df_curr['amount'].sum()
    df_curr = df_curr.loc[[idx],:].copy()
    df_curr['amount'] = amount_sum
    df_curr['source'] = 'all_sources'


    df = pd.concat([df_other, df_curr], axis=0).sort_values(by='date', ascending=True).reset_index(drop=True)

    return df


def calculate_sp500_returns(my_df: pd.DataFrame) -> pd.DataFrame:
    start_date = my_df['date'].min()
    end_date = my_df['date'].max()
    sp500_prices = get_sp500_data(start_date, end_date)
    sp500_prices.index = pd.to_datetime(sp500_prices.index, errors='coerce')

    transactions = []
    total_shares = 0.0
    for _, rows in my_df.iterrows():
        date = rows['date']
        amount = rows['amount']
        available_dates = sp500_prices.index
        closest_date = min(available_dates, key=lambda d: abs(d - date))
        px = sp500_prices.loc[closest_date, 'Close'].astype(float)
        if rows['desc'] == 'current value':
            transactions.append({
                'date': date.date(),
                'type': 'current value',
                'qty': total_shares,
                'px': px,
                'amount': round(px * total_shares, 1),
                'cum_qty': total_shares,
                'market_value': round(px * total_shares)
            })
        elif amount < 0:
            shares_bought = abs(amount) / px
            total_shares += shares_bought
            transactions.append({
                'date': date.date(),
                'type': 'buy',
                'qty': shares_bought,
                'px': px,
                'amount': amount,
                'cum_qty': total_shares,
                'market_value': px * total_shares
            })
        else:
            shares_sold = amount / px
            total_shares -= shares_sold
            transactions.append({
                'date': date.date(),
                'type': 'sell',
                'qty': shares_sold,
                'px': px,
                'amount': amount,
                'cum_qty': total_shares,
                'market_value': px * total_shares            
            })

    sp500_df = pd.DataFrame(transactions)
    return sp500_df

def calculate_irr(df: pd.DataFrame) -> float:
    returns = pyxirr.xirr(dict(zip(df['date'], df['amount'])))
    assert returns is not None
    return returns * 100

def total_investment(df: pd.DataFrame) -> float:
    total = df[df['desc'] != 'current value']['amount'].sum() * -1
    return total

def current_value(df: pd.DataFrame) -> float:
    return df[df['desc'] == 'current value']['amount'].sum()

def investment_summary() -> list[pd.DataFrame]:
    my_df = calculate_my_returns()
    sp500_df = calculate_sp500_returns(my_df)

    total_invest = total_investment(my_df)
    market_value = current_value(my_df)
    pnl = market_value - total_invest
    pnl_perc = pnl / total_invest * 100

    my_irr = calculate_irr(my_df.groupby('date').agg({'amount': 'sum', 'cumulative_investment': 'last'}).reset_index())
    sp500_irr = calculate_irr(sp500_df.groupby('date').agg({'amount': 'sum', 'market_value': 'last'}).reset_index())

    sp500_equivalent_market_value = sp500_df[sp500_df['type'] == 'current value'].loc[:, 'market_value'].iloc[0]
    sp500_equivalent_shares = sp500_df[sp500_df['type'] == 'current value'].loc[:, 'cum_qty'].iloc[0]
    beat_sp500_perc = (market_value - sp500_equivalent_market_value) / sp500_equivalent_market_value * 100

    print(f"Total investment: S$ {total_invest:,.0f}")
    print(f"Market value: S$ {market_value:,.0f}")
    print(f"PnL: S$ {pnl:,.2f}")
    print(f"Returns: {pnl_perc:.2f}%")
    print(f"IRR of my investments: {my_irr:.2f}%")
    print(f"IRR of sp500: {sp500_irr:.2f}%")

    print(f"S&P500 equivalent shares: {sp500_equivalent_shares:.2f}")
    print(f"S&P500 equivalent market value: S$ {sp500_equivalent_market_value:,.0f}")
    print(f"beat S&P500 by: {beat_sp500_perc:.2f}%")

    return [my_df, sp500_df]

def plot_investment_comparison(my_df: pd.DataFrame, sp500_df: pd.DataFrame):
    start_date = sp500_df['date'].min()
    end_date = my_df[my_df['desc'] == 'current value']['date'].iloc[0]
    sp500_prices = get_sp500_data(start_date, end_date)

    daily_values = pd.DataFrame(index=sp500_prices.index)
    daily_values['qty'] = np.nan

    # Calculate daily S&P500 shares and values
    for date in daily_values.index:
        # Get cumulative shares up to this date
        mask = sp500_df['date'] <= date
        if mask.any():
            daily_values.loc[date, 'qty'] = sp500_df[mask]['cum_qty'].iloc[-1]
        else:
            daily_values.loc[date, 'qty'] = 0

    daily_values['sp500_close'] = sp500_prices['Close']
    daily_values['sp500_value'] = daily_values['qty'] * daily_values['sp500_close']

    # Get final values
    final_portfolio_value = my_df[my_df['desc'] == 'current value']['amount'].iloc[0]
    final_sp500_value = sp500_df[sp500_df['type'] == 'current value']['amount'].iloc[0]

    # Plot
    import plotly.graph_objects as go
    fig = go.Figure()

    # Plot cumulative investment line (green)
    fig.add_trace(go.Scatter(
        x=my_df['date'],
        y=my_df['cumulative_investment'],
        mode='lines',
        name='Total Invested (S$)',
        line=dict(color='green')
    ))

    # Plot S&P500 mark-to-market value (red)
    fig.add_trace(go.Scatter(
        x=daily_values.index,
        y=daily_values['sp500_value'],
        mode='lines',
        name='S&P500 Mark-to-Market (S$)',
        line=dict(color='red')
    ))

    # Add marker for actual portfolio value (blue)
    fig.add_trace(go.Scatter(
        x=[end_date],
        y=[final_portfolio_value],
        mode='markers+text',
        name='Current Portfolio Value',
        text=[f'Portfolio: S${final_portfolio_value:,.0f}'],
        textposition='top right',
        marker=dict(color='blue', size=10)
    ))

    # Add marker for S&P500 final value (red)
    fig.add_trace(go.Scatter(
        x=[end_date],
        y=[final_sp500_value],
        mode='markers+text',
        name='S&P500 Final Value',
        text=[f'S&P500: S${final_sp500_value:,.0f}'],
        textposition='bottom right',
        marker=dict(color='red', size=10)
    ))

    fig.update_layout(
        title='Investment Growth Comparison',
        xaxis_title='Date',
        yaxis_title='Value (S$)',
        showlegend=True
    )
    
    fig.show()

# To add multiple values for an Enum, you can use tuples or assign aliases:
class Side(Enum):
    BUY = "buy"
    SELL = "sell"


class Holdings: 
    def __init__(self, ticker: str):
        self.ticker = ticker
        self.shares = 0
        self.sold_shares = 0
        self.bought_shares = 0
        self.market_value = 0
        self.market_px = get_last_close_px(self.ticker)
        self.spent = 0
        self.proceeds = 0
        self.cost_basis = 0
        self.cost_basis_per_share = 0
        self.proceeds_per_share = 0
        self.realised_pnl = 0
        self.unrealised_pnl = 0
        self.pnl = 0
        self.returns = 0

    def add_transaction(self, shares: float, px: float, side: Side):
        if side == Side.SELL:
            self.shares -= shares
            self.proceeds +=  shares * px
            self.sold_shares += shares
        else:
            self.shares += shares
            self.spent += shares * px
            self.bought_shares += shares


        self.cost_basis = (self.spent - self.proceeds)
        self.cost_basis_per_share = (self.spent - self.proceeds) / self.shares if self.shares > 0 else 0
        self.realised_pnl = self.proceeds - self.spent * (self.sold_shares / self.bought_shares)  if self.proceeds > 0 else 0
        self.unrealised_pnl = (self.market_px - self.cost_basis_per_share) * self.shares if self.shares > 0 else 0
        self.pnl = self.realised_pnl + self.unrealised_pnl
        self.market_value = self.market_px * self.shares
        self.returns = (self.pnl / self.spent) * 100 if self.spent > 0 else 0


def get_last_close_px(ticker: str) -> float:
    data = yf.download(ticker, period='1d', interval='1d', progress=False, auto_adjust=True)
    assert data is not None, f"No data found for ticker: {ticker}"
    if data.empty:
        raise ValueError(f"No data found for ticker: {ticker}")
    
    return data['Close'].iloc[-1].item()


def get_daily_change_pct(ticker: str) -> float:
    data = yf.download(ticker, period='2d', interval='1d', progress=False, auto_adjust=True)
    assert data is not None, f"No data found for ticker: {ticker}"
    if data.empty:
        raise ValueError(f"No data found for ticker: {ticker}")
    
    last_close = data['Close'].iloc[-1].item()
    previous_close = data['Close'].iloc[-2].item()
    
    return ((last_close - previous_close) / previous_close) * 100


def get_usdsgd_rate() -> float:
    return get_last_close_px('USDSGD=X')


def find_sum_of_columns(df):
    totals = pd.DataFrame({
        'ticker': ['Total'],
        'shares': [np.nan],
        'cost_basis': [df['cost_basis'].sum()],
        'cost_basis_per_share': [np.nan],
        'market_px': [np.nan],
        'market_value': [df['market_value'].sum()],
        'realised_pnl': [df['realised_pnl'].sum()],
        'unrealised_pnl': [df['unrealised_pnl'].sum()],
        'pnl': [df['pnl'].sum()],
        'returns': [(df['pnl'].sum() / df['cost_basis'].sum()) * 100]
    })
    df = pd.concat([df, totals], ignore_index=True)
    return df


def add_daily_change_pct(df: pd.DataFrame) -> pd.DataFrame:
    df['daily_change_pct'] = df['ticker'].apply(lambda x: get_daily_change_pct(x) if isinstance(x, str) else np.nan)
    return df


def calculate_stock_holdings(df: pd.DataFrame, sort_col: str) -> pd.DataFrame:
    holdings = {ticker: Holdings(ticker) for ticker in df['ticker'].unique()}
    for index, row in df.iterrows():
        ticker = row['ticker']
        holdings[ticker].add_transaction(row['qty'], row['px'], Side(row['side'].lower()))

    usdsgd_rate = get_usdsgd_rate()

    df = pd.DataFrame({
        'ticker': [h.ticker for h in holdings.values()],
        'shares': [h.shares for h in holdings.values()],
        'cost_basis': [h.cost_basis * usdsgd_rate for h in holdings.values()],
        'cost_basis_per_share': [round(h.cost_basis_per_share, 2) for h in holdings.values()],
        'market_px': [round(h.market_px, 2) for h in holdings.values()],
        'market_value': [round(h.market_px * h.shares * usdsgd_rate, 2) for h in holdings.values()],
        'realised_pnl': [round(h.realised_pnl * usdsgd_rate, 2) for h in holdings.values()],
        'unrealised_pnl': [round(h.unrealised_pnl * usdsgd_rate, 2) for h in holdings.values()],
        'pnl': [round(h.pnl * usdsgd_rate, 2) for h in holdings.values()],
        'returns': [round(h.returns * usdsgd_rate, 2) for h in holdings.values()]})
    
    df.sort_values(by=sort_col, ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = add_daily_change_pct(df)
    return find_sum_of_columns(df)


def pretty_print(df):
    def color_negative_red(val):
        """
        Colors negative values red and positive values green
        """
        if pd.isna(val):
            return ''
        try:
            # Handle both numeric and percentage string formats
            if isinstance(val, str):
                value = float(val.strip('%').replace(',', ''))
            else:
                value = float(val)
            color = 'red' if value < 0 else 'green'
            return f'background-color: {color}; color: white'
        except:
            return ''

    return df.style.format({
        'cost_basis': 'S${:,.2f}',
        'cost_basis_per_share': '${:,.2f}',
        'market_px': '${:,.2f}',
        'market_value': 'S${:,.2f}',
        'realised_pnl': 'S${:,.2f}',
        'unrealised_pnl': 'S${:,.2f}',
        'pnl': 'S${:,.2f}',
        'returns': '{:,.2f}%',
        'shares': '{:,.0f}',
        'daily_change_pct': '{:,.2f}%'
    }).set_properties(**{
        'text-align': 'right',
        'padding': '5px'
    }).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]},
        {'selector': '', 'props': [('border', '1px solid #ddd')]},
        {'selector': 'tbody tr:last-child', 'props': [('font-weight', 'bold'), ('border-top', '2px solid black')]}
    ]).applymap(color_negative_red, subset=['daily_change_pct', 'returns'])    

In [5]:
my_df, sp500_df = investment_summary()

Total investment: S$ 489,126
Market value: S$ 663,004
PnL: S$ 173,878.00
Returns: 35.55%
IRR of my investments: 18.25%
IRR of sp500: 15.27%
S&P500 equivalent shares: 100.86
S&P500 equivalent market value: S$ 629,185
beat S&P500 by: 5.38%


In [6]:
plot_investment_comparison(my_df, sp500_df)

In [85]:
stocks = pd.read_excel(STOCK_FILE, sheet_name="stocks")
stock_returns = calculate_stock_holdings(stocks, "returns")
pretty_print(stock_returns)

C:\Users\vasan\AppData\Local\Temp\ipykernel_31100\3313768013.py:386: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.



,ticker,shares,cost_basis,cost_basis_per_share,market_px,market_value,realised_pnl,unrealised_pnl,pnl,returns,daily_change_pct
0,PLTR,152,"S$4,758.47",$24.30,$154.27,"S$30,211.75",S$0.00,"S$25,453.27","S$25,453.27",689.17%,-2.58%
1,META,47,"S$18,785.90",$310.23,$750.01,"S$45,416.71",S$0.00,"S$26,630.80","S$26,630.80",182.64%,-3.03%
2,MSFT,48,"S$15,622.75",$252.62,$524.11,"S$32,412.64",S$0.00,"S$16,789.89","S$16,789.89",138.47%,-1.76%
3,AAPL,20,"S$2,931.11",$113.75,$202.38,"S$5,214.93",S$0.00,"S$2,283.82","S$2,283.82",100.39%,-2.50%
4,NVDA,356,"S$45,877.94",$100.02,$173.72,"S$79,680.23",S$0.00,"S$33,802.28","S$33,802.28",94.93%,-2.33%
5,TSLA,114,"S$29,899.90",$203.57,$302.63,"S$44,449.57",S$0.00,"S$14,549.67","S$14,549.67",62.70%,-1.83%
6,D05.SI,320,"S$13,724.81",$33.29,$47.60,"S$19,624.91",S$0.00,"S$5,900.10","S$5,900.10",55.39%,-0.65%
7,FBTC,440,"S$39,641.44",$69.93,$98.66,"S$55,929.96",S$0.00,"S$16,288.52","S$16,288.52",52.94%,-3.12%
8,O39.SI,500,"S$7,839.91",$12.17,$16.79,"S$10,816.12",S$0.00,"S$2,976.20","S$2,976.20",48.91%,-0.47%
9,GOOG,20,"S$3,846.13",$149.26,$189.95,"S$4,894.63",S$0.00,"S$1,048.50","S$1,048.50",35.12%,-1.51%


In [87]:
stocks[stocks['ticker'] == "NVDA"]

,Unnamed: 0,ticker,side,px,qty,broker
30,2024-08-02,NVDA,BUY,100.98,12,tiger
37,2024-09-09,NVDA,BUY,101.00,12,tiger
38,2024-09-09,NVDA,BUY,105.00,10,tiger
44,2024-10-02,NVDA,BUY,116.00,16,tiger
47,2024-11-05,NVDA,BUY,138.90,5,IBKR
63,2025-03-31,NVDA,BUY,105.00,31,tiger
66,2025-04-03,NVDA,BUY,102.50,30,tiger
69,2025-04-04,NVDA,BUY,97.00,65,tiger
72,2025-04-08,NVDA,BUY,96.00,65,IBKR
75,2025-04-16,NVDA,BUY,101.00,30,tiger
